In [1]:
import random
import numpy as np

from functools import reduce
from scipy.special import comb
binom = lambda n, k: comb(n, k, exact=True)

In [2]:
Q = 2657003489534545107915232808830590043

## Reference values

In [3]:
v = np.arange(27).reshape(3,3,3)
w = np.arange(27).reshape(3,3,3)

## Tensor of public values

In [4]:
class PublicTensor:
    
    def __init__(self, values):
        self.values = values
    
    def reveal(self):
        return self
    
    def unwrap(self):
        return self.values
    
    def add(x, y):
        if isinstance(y, PublicTensor):
            values = (x.values + y.values) % Q
            return PublicTensor(values)
        if isinstance(y, PrivateTensor):
            shares0 = (x.values + y.shares0) % Q
            shares1 =             y.shares1
            return PrivateTensor(None, shares0, shares1)
        
    def sub(x, y):
        if isinstance(y, PublicTensor):
            values = (x.values - y.values) % Q
            return PublicTensor(values)
        if isinstance(y, PrivateTensor):
            shares0 = (x.values + Q - y.shares0) % Q
            shares1 = (           Q - y.shares1) % Q
            return PrivateTensor(None, shares0, shares1)
        
    def mul(x, y):
        if isinstance(y, PublicTensor):
            values = (x.values * y.values) % Q
            return PublicTensor(values)
        if isinstance(y, PrivateTensor):
            shares0 = (x.values * y.shares0) % Q
            shares1 = (x.values * y.shares1) % Q
            return PrivateTensor(None, shares0, shares1)
    
    def dot(x, y):
        if isinstance(y, PublicTensor): 
            values = x.values.dot(y.values) % Q
            return PublicTensor(values)
        if isinstance(y, PrivateTensor):
            shares0 = x.values.dot(y.shares0) % Q
            shares1 = x.values.dot(y.shares1) % Q
            return PrivateTensor(None, shares0, shares1)
    
    def pows(x, highest_power):
        x_powers = ( np.power(x.values, e) % Q for e in range(0, highest_power+1) )
        return [ PublicTensor(v) for v in x_powers ]
    
    def __add__(x, y):
        return x.add(y)
    
    def __sub__(x, y):
        return x.sub(y)
    
    def __mul__(x, y):
        return x.mul(y)
    
    @property
    def shape(self):
        return self.values.shape
    
    def __repr__(self):
        return "PublicTensor(\n%s)" % self.values

In [5]:
x = PublicTensor(v)
y = PublicTensor(w)

z = x + y; assert (z.unwrap() == v + w).all()
z = x - y; assert (z.unwrap() == v - w).all()
z = x * y; assert (z.unwrap() == v * w).all()
z = x.dot(y); assert (z.unwrap() == v.dot(w)).all()

z = x.pows(2); 
assert (z[0].unwrap() == v**0).all
assert (z[1].unwrap() == v**1).all
assert (z[2].unwrap() == v**2).all

## Tensor of private values

In [6]:
def sample_random_tensor(shape):
    values = [ random.randrange(Q) for _ in range(np.prod(shape)) ]
    return np.array(values).reshape(shape)

In [7]:
def generate_mul_triple(x_shape, y_shape):
    a = sample_random_tensor(x_shape)
    b = sample_random_tensor(y_shape)
    c = np.multiply(a, b) % Q
    return PrivateTensor(a), PrivateTensor(b), PrivateTensor(c)

In [8]:
def generate_dot_triple(x_shape, y_shape):
    a = sample_random_tensor(x_shape)
    b = sample_random_tensor(y_shape)
    c = np.dot(a, b) % Q
    return PrivateTensor(a), PrivateTensor(b), PrivateTensor(c)

In [9]:
# TODO
#  - use eg repeated squaring instead, but NumPy doesn't support that
#  - avoid resharing '1' every time

def generate_pows_triple(shape, highest_power):
    a = random_values(shape)
#     a = np.zeros(shape).astype('int').astype('object')
    a_powers = ( np.power(a, e) % Q for e in range(0, highest_power+1) )
    return [ PrivateTensor(v) for v in a_powers ]

In [10]:
class PrivateTensor:
    
    def __init__(self, values, shares0=None, shares1=None):
        if not values is None:
            shares0 = sample_random_tensor(x.shape)
            shares1 = (values - shares0) % Q
        self.shares0 = shares0
        self.shares1 = shares1
    
    def reconstruct(self):
        return PublicTensor((self.shares0 + self.shares1) % Q)
    
    def unwrap(self):
        return (self.shares0 + self.shares1) % Q
    
    def add(x, y):
        if type(y) is PublicTensor:
            shares0 = (x.shares0 + y.values) % Q
            shares1 =  x.shares1
            return PrivateTensor(None, shares0, shares1)
        if type(y) is PrivateTensor:
            shares0 = (x.shares0 + y.shares0) % Q
            shares1 = (x.shares1 + y.shares1) % Q
            return PrivateTensor(None, shares0, shares1)
    
    def sub(x, y):
        if type(y) is PublicTensor:
            shares0 = (x.shares0 - y.values) % Q
            shares1 =  x.shares1
            return PrivateTensor(None, shares0, shares1)
        if type(y) is PrivateTensor:
            shares0 = (x.shares0 - y.shares0) % Q
            shares1 = (x.shares1 - y.shares1) % Q
            return PrivateTensor(None, shares0, shares1)
           
    def mul(x, y):
        if type(y) is PublicTensor:
            shares0 = (x.shares0 * y.values) % Q
            shares1 = (x.shares1 * y.values) % Q
            return PrivateTensor(None, shares0, shares1)
        if type(y) is PrivateTensor:
            a, b, c = generate_mul_triple(x.shape, y.shape)
            alpha = (x - a).reconstruct()
            beta  = (y - b).reconstruct()
            return alpha.mul(beta) + \
                   alpha.mul(b) + \
                   a.mul(beta) + \
                   c
                    
    def dot(x, y):
        if type(y) is PublicTensor:
            shares0 = x.shares0.dot(y.values) % Q
            shares1 = x.shares1.dot(y.values) % Q
            return PrivateTensor(None, shares0, shares1)
        if type(y) is PrivateTensor:
            a, b, c = generate_dot_triple(x.shape, y.shape)
            alpha = (x - a).reconstruct()
            beta  = (y - b).reconstruct()
            return alpha.dot(beta) + \
                   alpha.dot(b) + \
                   a.dot(beta) + \
                   c
    
    def conv2d(x, y):
        pass
    
    def pows(x, highest_power):
        assert highest_power >= 1        
        a_powers = generate_pows_triple(x.shape, highest_power)
        one = a_powers[0]
        a = a_powers[1]
        alpha = (x - a).reveal()
        alpha_powers = alpha.pows(highest_power)
        x_powers = []
        for power in range(1, highest_power+2):
            # compute binomial coefficients
            coeffs = [ PublicTensor(binom(power, k)) for k in range(power+1) ]
            # compute and sum terms
            foo = a_powers[:power+1]
            bar = list(reversed(alpha_powers[:power]))
            print(len(foo), len(bar))
            terms = [
                a_power * (alpha_power * coeff) \
                for a_power, alpha_power, coeff in zip (\
                    foo,
                    bar,
                    coeffs
                )
            ]
            x_powers.append(reduce(lambda x, y: x + y, terms))
        return x_powers
        
    def __add__(x, y):
        return x.add(y)
            
    def __sub__(x, y):
        return x.sub(y)
    
    def __mul__(x, y):
        return x.mul(y)
    
    @property
    def shape(self):
        return self.shares0.shape
    
    def __repr__(self):
        return "PrivateTensor(\n%s)" % reconstruct(self.shares0, self.shares1)

In [11]:
x = PrivateTensor(v)
y = PrivateTensor(w)

z = x + y; assert (z.unwrap() == v + w).all()
z = x - y; assert (z.unwrap() == v - w).all()
z = x * y; assert (z.unwrap() == v * w).all()
z = x.dot(y); assert (z.unwrap() == v.dot(w)).all()

# z = x.pows(2);
# print(len(z), z)
# assert (z[0].unwrap() == v**0).all()
# assert (z[1].unwrap() == v**1).all()
# assert (z[2].unwrap() == v**2).all()